In [1]:
import pandas as pd
import os
import glob
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
from skfda.representation.grid import FDataGrid
import fdasrsf as fs


import skfda
from skfda.datasets import fetch_growth
from skfda.exploratory.visualization import FPCAPlot
from skfda.preprocessing.dim_reduction import FPCA
from skfda.representation.basis import (
    BSplineBasis,
    FourierBasis,
    MonomialBasis,
)


from __future__ import annotations

from typing import Any, Mapping, Tuple

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import sklearn.cluster
from cartopy.io.img_tiles import GoogleTiles
from matplotlib.axes import Axes
from matplotlib.figure import Figure

from skfda.datasets import fetch_aemet
from skfda.exploratory.depth import ModifiedBandDepth
from skfda.exploratory.visualization import Boxplot, MagnitudeShapePlot
from skfda.exploratory.visualization.fpca import FPCAPlot
from skfda.misc.metrics import l2_distance
from skfda.ml.clustering import KMeans
from skfda.preprocessing.dim_reduction import FPCA

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from kneed import KneeLocator


from sklearn.datasets import fetch_openml
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import CondensedNearestNeighbour, TomekLinks
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from IPython.display import Image

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Data Cleaning and Time Warping

In [2]:
#read each csv file in current_directory and store it in dataframes

current_directory = os.getcwd()
csv_files = glob.glob(os.path.join(current_directory, 'ABR_exported_files/*.csv'))

dataframes = {}
for file in csv_files:
    filename = os.path.splitext(os.path.basename(file))[0]
    if pd.read_csv(file).shape[1] > 1:
        dataframes[filename] = pd.read_csv(file)
    else:
        dataframes[filename] = pd.read_csv(file, skiprows=2)

In [3]:
subjects = [0,1,2,3,4,5,6,7,8,9,10]
frequencies = [4000, 8000, 16000, 24000, 32000]
dB_levels = [20,25,30,35,40,45,50,55,60,65,70,75,80,85,90]
results_df = pd.DataFrame()

# Iterate over all subject-frequency pairs
for subject in subjects:
    for frequency in frequencies:
        waves = []
        available_dB_levels = []  # List to store the available dB levels for this subject/frequency pairing
        for dB in dB_levels:
            filename = list(dataframes.items())[subject][0]
            df = list(dataframes.values())[subject]
            khz = df[df['Freq(Hz)'] == frequency]
            dbkhz = khz[khz['Level(dB)'] == dB]  # Select the dB level
            if not dbkhz.empty:
                index = dbkhz.index.values[0]
                final = df.iloc[index,48:]
                final = pd.to_numeric(final, errors='coerce')
                wave = final.tolist()[:-1]
                waves.append(wave)
                available_dB_levels.append(dB)  # Add the dB level to the list of available dB levels

        # Align the waves using dynamic time warping
        np_waves = np.array(waves)
        time = np.linspace(0, 1, np_waves.shape[1])
        obj = fs.fdawarp(np_waves.T, time)
        obj.srsf_align(parallel=True)
        aligned_waves = obj.fn.T

        # Iterate over all available dB levels to find the peaks in the aligned waves
        for i, dB in enumerate(available_dB_levels):
            wave = aligned_waves[i]
            wave_after_30 = wave[50:-30]  # start from the 30th point and end before the last 30 points
            peaks, _ = find_peaks(wave_after_30, distance=15)
            peaks = [peak + 50 for peak in peaks]  # adjust the indices
            highest_peaks_indices = sorted(peaks, key=lambda x: wave[x], reverse=True)[:5]
            highest_peaks_indices.sort()
            peak_coordinates = []
            for peak in highest_peaks_indices:
                peak_coordinates.extend([peak, wave[peak]])

            # If fewer than 5 peaks, fill the remaining coordinates with None
            while len(peak_coordinates) < 10:
                peak_coordinates.extend([None, None])

            # Add the results to the dataframe
            new_row = pd.DataFrame([{'Subject': filename, 'Frequency': frequency, 'dB': dB, 'Peak1_X': peak_coordinates[0], 'Peak1_Y': peak_coordinates[1], 'Peak2_X': peak_coordinates[2], 'Peak2_Y': peak_coordinates[3], 'Peak3_X': peak_coordinates[4], 'Peak3_Y': peak_coordinates[5], 'Peak4_X': peak_coordinates[6], 'Peak4_Y': peak_coordinates[7], 'Peak5_X': peak_coordinates[8], 'Peak5_Y': peak_coordinates[9]}])
            results_df = pd.concat([results_df, new_row], ignore_index=True)


Initializing...


/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas re

Compute Karcher Mean of 8 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 14 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
I

/var/folders/g_/b1twxg5159dc5n5_hm9nxdgm0000gn/T/ipykernel_50297/2423822563.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


Initializing...
Compute Karcher Mean of 15 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 8 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterati

/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homebrew/anaconda3/lib/python3.11/site-packages/fdasrsf/utility_functions.py:168: RuntimeWarning: invalid value encountered in divide
  gam = orN2.coptimum_reparam(ascontiguousarray(q1), time,
/opt/homeb

updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 15 function in SRSF space with lam=0
updating step: r=1
updating step: r=2
updating step: r=3
updating step: r=4
updating step: r=5
updating step: r=6
updating step: r=7
updating step: r=8
updating step: r=9
updating step: r=10
updating step: r=11
updating step: r=12
updating step: r=13
updating step: r=14
updating step: r=15
updating step: r=16
updating step: r=17
updating step: r=18
updating step: r=19
updating step: r=20
maximal number of iterations is reached
Initializing...
Compute Karcher Mean of 15 function in SRSF space with lam=0
upd

In [4]:
# remove rows with NA
results_df = results_df[~results_df['Subject'].str.startswith('A')]

#add deaf column
def check_subject(subject):
    if subject.startswith(('B1_1285', 'B2_1291', 'B2_1290')):
        return 1
    else:
        return 0

results_df['Deaf'] = results_df['Subject'].apply(check_subject)

In [5]:
results_df

,Subject,Frequency,dB,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf
0,B1_1282_baseline,4000,55,81,0.001650,112,0.001422,136,0.001335,152,-0.000050,180,0.001415,0
1,B1_1282_baseline,4000,60,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,0.003407,0
2,B1_1282_baseline,4000,65,81,0.007780,112,-0.001040,136,0.003537,151,0.000362,191,0.000339,0
3,B1_1282_baseline,4000,70,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,0.001529,0
4,B1_1282_baseline,4000,75,81,0.007113,112,0.002525,136,0.006475,151,0.005110,191,0.009667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,B2_1292_baseline,32000,70,79,0.009000,100,0.006000,117,0.019526,169,0.008000,201,0.010877,0
506,B2_1292_baseline,32000,75,79,0.008000,117,0.027383,139,0.007000,169,0.010000,201,0.011000,0
507,B2_1292_baseline,32000,80,79,0.013000,117,0.030194,139,0.009000,169,0.013000,200,0.006000,0
508,B2_1292_baseline,32000,85,58,0.013000,79,0.010292,117,0.031957,169,0.015000,201,0.016877,0


In [6]:
# Define the dB thresholds for each subject and frequency
thresholds = {
    'B1': {
        4000: {'1284': 70, '1285': 90, '1286': 80, '1287': 70, '1279': 70, '1280': 80, '1281': 90, '1282': 75},
        8000: {'1284': 45, '1285': 70, '1286': 50, '1287': 30, '1279': 30, '1280': 50, '1281': 65, '1282': 50},
        16000: {'1284': 45, '1285': 60, '1286': 35, '1287': 20, '1279': 25, '1280': 35, '1281': 50, '1282': 35},
        24000: {'1284': 45, '1285': 65, '1286': 40, '1287': 35, '1279': 35, '1280': 40, '1281': 55, '1282': 40},
        32000: {'1284': 50, '1285': 60, '1286': 35, '1287': 30, '1279': 35, '1280': 35, '1281': 70, '1282': 45}
    },
    'B2': {
        4000: {'1283': 70, '1291': 90, '1292': 80, '1290': 80, '1288': 60, '1289': 80},
        8000: {'1283': 30, '1291': 85, '1292': 60, '1290': 75, '1288': 45, '1289': 40},
        16000: {'1283': 20, '1291': 60, '1292': 40, '1290': 60, '1288': 30, '1289': 25},
        24000: {'1283': 35, '1291': 65, '1292': 40, '1290': 70, '1288': 35, '1289': 35},
        32000: {'1283': 35, '1291': 75, '1292': 35, '1290': 90, '1288': 50, '1289': 35}
    }
}

# Add the 'signal' column
def check_signal(row):
    group = row['Subject'].split('_')[0]
    subject = row['Subject'].split('_')[1]
    frequency = row['Frequency']
    dB = row['dB']
    
    # Check if any of the values are nan
    if pd.isnull(group) or pd.isnull(subject) or pd.isnull(frequency) or pd.isnull(dB):
        return np.nan  # or return some other value that makes sense in your context
    
    if dB >= thresholds[group][frequency][subject]:
        return 1
    else:
        return 0


results_df['Signal'] = results_df.apply(check_signal, axis=1)

results_df.head()


,Subject,Frequency,dB,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf,Signal
0,B1_1282_baseline,4000,55,81,0.001650,112,0.001422,136,0.001335,152,-0.000050,180,0.001415,0,0
1,B1_1282_baseline,4000,60,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,0.003407,0,0
2,B1_1282_baseline,4000,65,81,0.007780,112,-0.001040,136,0.003537,151,0.000362,191,0.000339,0,0
3,B1_1282_baseline,4000,70,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,0.001529,0,0
4,B1_1282_baseline,4000,75,81,0.007113,112,0.002525,136,0.006475,151,0.005110,191,0.009667,0,1


In [7]:
encoded = pd.get_dummies(results_df, columns=['Frequency'])
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoded['dB_encoded'] = encoder.fit_transform(encoded[['dB']])
encoded = encoded.drop('dB', axis=1)


In [8]:
encoded

,Subject,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf,Signal,Frequency_4000,Frequency_8000,Frequency_16000,Frequency_24000,Frequency_32000,dB_encoded
0,B1_1282_baseline,81,0.001650,112,0.001422,136,0.001335,152,-0.000050,180,0.001415,0,0,True,False,False,False,False,7.0
1,B1_1282_baseline,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,0.003407,0,0,True,False,False,False,False,8.0
2,B1_1282_baseline,81,0.007780,112,-0.001040,136,0.003537,151,0.000362,191,0.000339,0,0,True,False,False,False,False,9.0
3,B1_1282_baseline,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,0.001529,0,0,True,False,False,False,False,10.0
4,B1_1282_baseline,81,0.007113,112,0.002525,136,0.006475,151,0.005110,191,0.009667,0,1,True,False,False,False,False,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,B2_1292_baseline,79,0.009000,100,0.006000,117,0.019526,169,0.008000,201,0.010877,0,1,False,False,False,False,True,10.0
506,B2_1292_baseline,79,0.008000,117,0.027383,139,0.007000,169,0.010000,201,0.011000,0,1,False,False,False,False,True,11.0
507,B2_1292_baseline,79,0.013000,117,0.030194,139,0.009000,169,0.013000,200,0.006000,0,1,False,False,False,False,True,12.0
508,B2_1292_baseline,58,0.013000,79,0.010292,117,0.031957,169,0.015000,201,0.016877,0,1,False,False,False,False,True,13.0


# Train/Test Split

In [9]:
#train/test split
from sklearn.model_selection import train_test_split
encoded = encoded.dropna()

feature_cols = [col for col in encoded.columns if col not in ['Subject', 'Signal']]
X = encoded[feature_cols]
y = encoded['Signal']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)


# Scaling

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_cols = ['Peak1_X', 'Peak1_Y', 'Peak2_X', 'Peak2_Y', 'Peak3_X', 'Peak3_Y', 'Peak4_X', 'Peak4_Y', 'Peak5_X', 'Peak5_Y']
scaler.fit(x_train[numeric_cols])
x_train[numeric_cols] = scaler.transform(x_train[numeric_cols])
x_test[numeric_cols] = scaler.transform(x_test[numeric_cols])


In [11]:
x_train

,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,Peak5_Y,Deaf,Frequency_4000,Frequency_8000,Frequency_16000,Frequency_24000,Frequency_32000,dB_encoded
342,-0.639254,-0.544755,1.011871,-0.621466,0.807247,-0.190609,1.095929,0.036314,0.304575,-0.013983,0,False,False,True,False,False,1.0
355,-0.639254,2.696394,0.940550,7.993838,0.807247,3.038045,1.155507,0.499213,0.304575,3.717099,0,False,False,True,False,False,14.0
423,0.612333,-0.270396,0.869229,-0.689121,0.652943,-0.621889,0.440572,-0.877283,-0.147472,-0.617067,1,False,False,False,False,True,3.0
431,0.612333,-0.292002,-0.129270,-0.380328,0.730095,-0.621889,0.261838,-0.886318,-0.147472,-0.617067,1,False,False,False,False,True,11.0
285,-0.281658,0.092630,-0.628519,1.050120,-0.350033,0.126726,0.738462,0.220133,0.982645,-0.031196,1,False,False,True,False,False,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,-0.192259,0.092630,0.013373,0.308143,-0.272881,0.353985,-0.453097,2.772966,-1.578953,-0.910002,0,True,False,False,False,False,14.0
74,-1.086249,-0.529757,-0.557198,-0.429219,-0.735793,0.290409,-1.048876,1.180412,-1.352930,1.975889,0,True,False,False,False,False,12.0
501,-0.371057,0.092630,-1.056447,-0.483259,-0.427185,-0.446570,0.917195,0.220133,0.605940,-0.324131,0,False,False,False,False,True,6.0
162,-0.192259,-0.270396,-0.271913,0.596196,-0.427185,0.188348,-0.691408,0.932173,-1.202248,-0.324131,0,False,False,True,False,False,8.0


# Logistic Regression 

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

log_reg = LogisticRegression(C=1e10)
log_reg.fit(x_train, y_train)
y_pred = log_reg.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.90      0.87      0.89        31
           1       0.93      0.95      0.94        58

    accuracy                           0.92        89
   macro avg       0.92      0.91      0.91        89
weighted avg       0.92      0.92      0.92        89



# confusion matrix

In [16]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, columns=['Predicted: No', 'Predicted: Yes'], index=['Actual: No', 'Actual: Yes'])
print(df_cm)

             Predicted: No  Predicted: Yes
Actual: No              27               4
Actual: Yes              3              55


# Getting Thresholds

In [ ]:
# Preprocess the whole dataset
X_whole = encoded[feature_cols]
X_whole[numeric_cols] = scaler.transform(X_whole[numeric_cols])

# Make predictions on the whole dataset
y_whole_pred = log_reg.predict(X_whole)

# Create a new dataframe that includes the predictions and the 'Subject'
encoded_with_predictions = encoded.copy()
encoded_with_predictions['Predictions'] = y_whole_pred

# Now, 'encoded_with_predictions' is your original dataframe with an additional column for the predictions.


/var/folders/g_/b1twxg5159dc5n5_hm9nxdgm0000gn/T/ipykernel_61450/1256325110.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_whole[numeric_cols] = scaler.transform(X_whole[numeric_cols])


In [ ]:
df = encoded_with_predictions

In [ ]:
df

,Subject,Peak1_X,Peak1_Y,Peak2_X,Peak2_Y,Peak3_X,Peak3_Y,Peak4_X,Peak4_Y,Peak5_X,...,Deaf,Signal,Frequency_4000,Frequency_8000,Frequency_16000,Frequency_24000,Frequency_32000,dB_encoded,Predictions,Smoothed Predictions
0,B1_1282_baseline,81,0.00165,112,0.001422,136,0.001335,152,-0.00005,180,...,0,0,True,False,False,False,False,7.0,0,NaN
1,B1_1282_baseline,66,-0.002112,81,0.002746,112,-0.000114,136,0.003893,191,...,0,0,True,False,False,False,False,8.0,0,NaN
2,B1_1282_baseline,81,0.00778,112,-0.00104,136,0.003537,151,0.000362,191,...,0,0,True,False,False,False,False,9.0,0,NaN
3,B1_1282_baseline,81,0.004333,135,0.002188,151,0.002845,191,0.002547,211,...,0,0,True,False,False,False,False,10.0,0,NaN
4,B1_1282_baseline,81,0.007113,112,0.002525,136,0.006475,151,0.00511,191,...,0,1,True,False,False,False,False,11.0,1,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,B2_1292_baseline,79,0.009,100,0.006,117,0.019526,169,0.008,201,...,0,1,False,False,False,False,True,10.0,1,1.0
506,B2_1292_baseline,79,0.008,117,0.027383,139,0.007,169,0.01,201,...,0,1,False,False,False,False,True,11.0,1,1.0
507,B2_1292_baseline,79,0.013,117,0.030194,139,0.009,169,0.013,200,...,0,1,False,False,False,False,True,12.0,1,1.0
508,B2_1292_baseline,58,0.013,79,0.010292,117,0.031957,169,0.015,201,...,0,1,False,False,False,False,True,13.0,1,1.0


In [ ]:
# Assuming 'df' is your DataFrame and 'Predictions' are your predictions
thresholds = []
for subject in df['Subject'].unique():
    for frequency in ['Frequency_4000', 'Frequency_8000', 'Frequency_16000', 'Frequency_24000', 'Frequency_32000']:
        subject_df = df[df['Subject'] == subject]
        freq_df = subject_df[subject_df[frequency] == True]
        
        # Sort by dB_encoded
        freq_df = freq_df.sort_values('dB_encoded')
        
        # Initialize variables
        last_prediction = freq_df.iloc[0]['Predictions']
        fuzziness_start = None
        fuzziness_end = None
        
        # Iterate over rows
        for _, row in freq_df.iterrows():
            if row['Predictions'] != last_prediction:
                if fuzziness_start is None:
                    fuzziness_start = row['dB_encoded']
                else:
                    fuzziness_end = row['dB_encoded']
                    
            last_prediction = row['Predictions']
        
        # Calculate midpoint of fuzziness
        if fuzziness_start is not None and fuzziness_end is not None:
            threshold = (fuzziness_start + fuzziness_end) / 2
        else:
            # Fall back to regular threshold if no fuzziness
            prediction_threshold = freq_df[freq_df['Predictions'] == 1]['dB_encoded'].min()
            threshold = prediction_threshold
        
        # Get the ground truth threshold
        ground_truth_threshold = freq_df[freq_df['Signal'] == 1]['dB_encoded'].min()
        
        thresholds.append([subject, frequency, threshold, ground_truth_threshold])

threshold_df = pd.DataFrame(thresholds, columns=['Subject', 'Frequency', 'Threshold', 'Ground Truth Threshold'])


In [ ]:
threshold_df

,Subject,Frequency,Threshold,Ground Truth Threshold
0,B1_1282_baseline,Frequency_4000,11.0,11.0
1,B1_1282_baseline,Frequency_8000,5.0,6.0
2,B1_1282_baseline,Frequency_16000,2.0,3.0
3,B1_1282_baseline,Frequency_24000,3.0,4.0
4,B1_1282_baseline,Frequency_32000,6.5,5.0
5,B2_1288_baseline,Frequency_4000,8.0,8.0
6,B2_1288_baseline,Frequency_8000,5.0,5.0
7,B2_1288_baseline,Frequency_16000,2.0,2.0
8,B2_1288_baseline,Frequency_24000,3.0,3.0
9,B2_1288_baseline,Frequency_32000,6.0,6.0
